## Towards quantum computing in the coupled basis

#### Imports

In [344]:
from typing import List
import numpy as np
from scipy.sparse import lil_matrix
from NSMFermions.nuclear_physics_utils import SingleParticleState,get_twobody_nuclearshell_model
from NSMFermions.hamiltonian_utils import FermiHubbardHamiltonian
from NSMFermions import QuasiParticlesConverterOnlynnpp,QuasiParticlesConverter
from scipy.sparse.linalg import eigsh

#### Read the CKI basis

In [385]:
# ==============================================================
# CKI parser with: j-only representation, T and Tz expansion,
# single-particle energies stored per (j_a,j_b) couple.
# ==============================================================

tbme_dict = {}        # key: ((j_ap,j_bp),(j_a,j_b),J,T,Tz) -> V
states_set = set()    # (j_a,j_b,J,T,Tz)
couple_spe = {}       # (j_a,j_b) -> epsilon_a + epsilon_b

def decode_j(nlj):
    """Decode CKI nlj to j (last digit / 2)."""
    j2 = int(nlj) % 10
    return j2 / 2

def is_header(line):
    """Detect TBME header lines: 8 integers."""
    parts = line.split()
    if len(parts) != 8:
        return False
    try:
        [int(p) for p in parts]
        return True
    except ValueError:
        return False


with open("data/cki", "r") as f:
    lines = f.readlines()



# ==============================================================
# Step 2 — TBMEs with expansion on Tz
# ==============================================================

i = 0
while i < len(lines):
    line = lines[i].strip()
    if is_header(line):
        parts = line.split()
        T_min, T_max = int(parts[0]), int(parts[1])
        j_a, j_b = decode_j(parts[2]), decode_j(parts[3])
        j_ap, j_bp = decode_j(parts[4]), decode_j(parts[5])
        J_min, J_max = int(parts[6]), int(parts[7])

        T_list = range(T_min, T_max+1)
        J_list = range(J_min, J_max+1)

        # read all numeric values in block
        values = []
        i += 1
        while i < len(lines):
            l = lines[i].strip()
            if l == "" or is_header(l):
                break
            for num in l.split():
                try:
                    values.append(float(num))
                except ValueError:
                    pass
            i += 1

        # Assign values to all (J,T) and expand to Tz
        idx = 0
        for T in T_list:
            for J in J_list:
                if idx < len(values):
                    V = values[idx]
                    idx += 1
                    if j_a==j_b:
                        n_ab=np.sqrt(1-(-1)**(J+T))/2
                    else:
                        n_ab=1
                    if j_ap==j_bp:
                        n_apbp=np.sqrt(1-(-1)**(J+T))/2
                    else:
                        n_apbp=1

                    for Tz in range(-T, T+1):
                        key = ((j_ap, j_bp), (j_a, j_b), J, T, Tz)
                        tbme_dict[key] =n_ab*n_apbp*V#*(2+2*(-1)**(J+T))

                        #tbme_dict[((j_bp, j_ap), (j_a, j_b), J, T, Tz)]=n_ab*n_apbp*V*(-1)**(J+T)
                        #tbme_dict[((j_ap, j_bp), (j_b, j_a), J, T, Tz)]=n_ab*n_apbp*V*(-1)**(J+T)
                        tbme_dict[((j_a, j_b), (j_ap, j_ap), J, T, Tz)]=n_ab*n_apbp*V#*(-1)**(J+T)
                        
                        
                                        # Also create entries for permutations that are related by the antisymmetry phase:
                    # <ba|V|cd> = (-1)^{J+T} <ab|V|cd>;  <ab|V|dc> = (-1)^{J+T} <ab|V|cd>
                        # register states (if phase / antisymmetry condition requires)
                        if n_ab*n_apbp*V!=0:
                            states_set.add((j_a, j_b, J, T, Tz))
                            states_set.add((j_ap, j_bp, J, T, Tz))
                            # states_set.add((j_bp, j_ap, J, T, Tz))
                            # states_set.add((j_b, j_a, J, T, Tz))

        continue

    i += 1

# ==============================================================
# Step 3 — build ordered list and SPE alignment
# ==============================================================

states_list = sorted(states_set)
spe_vector = []

for (j_a, j_b, J, T, Tz) in states_list:
    if (j_a, j_b) in couple_spe:
        spe_vector.append(couple_spe[(j_a, j_b)])
    elif (j_b, j_a) in couple_spe:
        spe_vector.append(couple_spe[(j_b, j_a)])
    else:
        spe_vector.append(0.0)

# ==============================================================
# Output
# ==============================================================

print("Coupled basis states and SPE:")
for s, e in zip(states_list, spe_vector):
    print(f"{s} -> {e}")

print("\nFirst 10 TBME entries:")
for k, v in list(tbme_dict.items())[:]:
    print(k, v)
    
    

# ---------------------------------------------
# Extract single-particle energies per j
# -------------
# ---------------------------------------------
# Parse SINGLE-PARTICLE ENERGIES — first block only
# ---------------------------------------------
spe_per_j = {}

i = 0
spe_found = False
while i < len(lines) and not spe_found:
    parts = lines[i].split()

    # Look for the *first* orbital line: 4 integers
    if len(parts) >= 4 and all(p.isdigit() for p in parts[:4]):
        j1 = decode_j(parts[2])   # 103 -> 3/2
        j2 = decode_j(parts[3])   # 101 -> 1/2

        # Next line contains the two SPE values
        if i+1 < len(lines):
            energy_line = lines[i+1].split()
            if len(energy_line) >= 2:
                e1 = float(energy_line[0])
                e2 = float(energy_line[1])

                spe_per_j[j1] = e1
                spe_per_j[j2] = e2

                spe_found = True   # IMPORTANT: read only first block
                break
    i += 1

print("Single-particle energies extracted from first block:")
print(spe_per_j)
spe_vector = []
for (j_a, j_b, J, Tz, T) in states_list:
    E_pair = spe_per_j.get(j_a, 0.0) + spe_per_j.get(j_b, 0.0)
    spe_vector.append(E_pair)
    
print(spe_vector)

Coupled basis states and SPE:
(0.5, 0.5, 0, 1, -1) -> 0.0
(0.5, 0.5, 0, 1, 0) -> 0.0
(0.5, 0.5, 0, 1, 1) -> 0.0
(0.5, 0.5, 1, 0, 0) -> 0.0
(1.5, 0.5, 1, 0, 0) -> 0.0
(1.5, 0.5, 1, 1, -1) -> 0.0
(1.5, 0.5, 1, 1, 0) -> 0.0
(1.5, 0.5, 1, 1, 1) -> 0.0
(1.5, 0.5, 2, 0, 0) -> 0.0
(1.5, 0.5, 2, 1, -1) -> 0.0
(1.5, 0.5, 2, 1, 0) -> 0.0
(1.5, 0.5, 2, 1, 1) -> 0.0
(1.5, 1.5, 0, 1, -1) -> 0.0
(1.5, 1.5, 0, 1, 0) -> 0.0
(1.5, 1.5, 0, 1, 1) -> 0.0
(1.5, 1.5, 1, 0, 0) -> 0.0
(1.5, 1.5, 2, 1, -1) -> 0.0
(1.5, 1.5, 2, 1, 0) -> 0.0
(1.5, 1.5, 2, 1, 1) -> 0.0
(1.5, 1.5, 3, 0, 0) -> 0.0

First 10 TBME entries:
((1.5, 1.5), (1.5, 1.5), 0, 0, 0) 0.0
((1.5, 1.5), (1.5, 1.5), 1, 0, 0) 2.84525626613843
((1.5, 1.5), (1.5, 1.5), 2, 0, 0) 0.0
((1.5, 1.5), (1.5, 1.5), 3, 0, 0) -3.338950000000001
((1.5, 1.5), (1.5, 1.5), 0, 1, -1) -1.3676000000000001
((1.5, 1.5), (1.5, 1.5), 0, 1, 0) -1.3676000000000001
((1.5, 1.5), (1.5, 1.5), 0, 1, 1) -1.3676000000000001
((1.5, 1.5), (1.5, 1.5), 1, 1, -1) 0.0
((1.5, 1.5), (1.5, 

In [386]:
print(states_list)

[(0.5, 0.5, 0, 1, -1), (0.5, 0.5, 0, 1, 0), (0.5, 0.5, 0, 1, 1), (0.5, 0.5, 1, 0, 0), (1.5, 0.5, 1, 0, 0), (1.5, 0.5, 1, 1, -1), (1.5, 0.5, 1, 1, 0), (1.5, 0.5, 1, 1, 1), (1.5, 0.5, 2, 0, 0), (1.5, 0.5, 2, 1, -1), (1.5, 0.5, 2, 1, 0), (1.5, 0.5, 2, 1, 1), (1.5, 1.5, 0, 1, -1), (1.5, 1.5, 0, 1, 0), (1.5, 1.5, 0, 1, 1), (1.5, 1.5, 1, 0, 0), (1.5, 1.5, 2, 1, -1), (1.5, 1.5, 2, 1, 0), (1.5, 1.5, 2, 1, 1), (1.5, 1.5, 3, 0, 0)]


#### Build up the many-body basis

Number of particles

In [397]:
# Desired total number of neutrons and protons
N_neutron = 2
N_proton = 2

In [398]:
from itertools import combinations

# Precompute nucleon contributions of each coupled state
pair_contrib = []
for (j_a, j_b, J, T, Tz) in states_list:
    if Tz == -1:
        pair_contrib.append((2, 0)) # 2 neutrons
    elif Tz == +1:
        pair_contrib.append((0, 2)) # 2 protons
    elif Tz == 0:
        pair_contrib.append((1, 1)) # 1 proton + 1 neutron

N = len(states_list)

valid_configs = []

# instead of all 2^N bitstrings, iterate only by number of occupied pairs
max_pairs = (N_neutron + N_proton) // 2  # cannot exceed this
for k in range(1, max_pairs+1):          # k = number of occupied pair states
    for comb in combinations(range(N), k):
        nn = np_number = 0
        for idx in comb:
            dn, dp = pair_contrib[idx]
            nn += dn
            np_number += dp
            # prune early if too many nucleons
            if nn > N_neutron or np_number > N_proton:
                break

        if nn == N_neutron and np_number == N_proton:
            # build a bitstring only when successful
            bits = tuple(1 if i in comb else 0 for i in range(N))
            valid_configs.append(bits)

print(f"Number of valid many-body states: {len(valid_configs)}")

Number of valid many-body states: 70


In [399]:
print(list(valid_configs[0]))

[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [400]:
for base in valid_configs:
    base=list(base)
    base=np.array(base)
    idx=np.nonzero(base)[0]
    for ind in idx:
        print(states_list[ind])
    print('\n')

(0.5, 0.5, 0, 1, -1)
(0.5, 0.5, 0, 1, 1)


(0.5, 0.5, 0, 1, -1)
(1.5, 0.5, 1, 1, 1)


(0.5, 0.5, 0, 1, -1)
(1.5, 0.5, 2, 1, 1)


(0.5, 0.5, 0, 1, -1)
(1.5, 1.5, 0, 1, 1)


(0.5, 0.5, 0, 1, -1)
(1.5, 1.5, 2, 1, 1)


(0.5, 0.5, 0, 1, 0)
(0.5, 0.5, 1, 0, 0)


(0.5, 0.5, 0, 1, 0)
(1.5, 0.5, 1, 0, 0)


(0.5, 0.5, 0, 1, 0)
(1.5, 0.5, 1, 1, 0)


(0.5, 0.5, 0, 1, 0)
(1.5, 0.5, 2, 0, 0)


(0.5, 0.5, 0, 1, 0)
(1.5, 0.5, 2, 1, 0)


(0.5, 0.5, 0, 1, 0)
(1.5, 1.5, 0, 1, 0)


(0.5, 0.5, 0, 1, 0)
(1.5, 1.5, 1, 0, 0)


(0.5, 0.5, 0, 1, 0)
(1.5, 1.5, 2, 1, 0)


(0.5, 0.5, 0, 1, 0)
(1.5, 1.5, 3, 0, 0)


(0.5, 0.5, 0, 1, 1)
(1.5, 0.5, 1, 1, -1)


(0.5, 0.5, 0, 1, 1)
(1.5, 0.5, 2, 1, -1)


(0.5, 0.5, 0, 1, 1)
(1.5, 1.5, 0, 1, -1)


(0.5, 0.5, 0, 1, 1)
(1.5, 1.5, 2, 1, -1)


(0.5, 0.5, 1, 0, 0)
(1.5, 0.5, 1, 0, 0)


(0.5, 0.5, 1, 0, 0)
(1.5, 0.5, 1, 1, 0)


(0.5, 0.5, 1, 0, 0)
(1.5, 0.5, 2, 0, 0)


(0.5, 0.5, 1, 0, 0)
(1.5, 0.5, 2, 1, 0)


(0.5, 0.5, 1, 0, 0)
(1.5, 1.5, 0, 1, 0)


(0.5, 0.5, 1, 0, 0)
(1.5,

In [401]:
def p_dag_p_matrix(a,b, basis):
    matrix=lil_matrix((len(basis), len(basis)), dtype=int)
    for idx, base in enumerate(basis):
        new_base=list(base)
        if a!=b:
            if base[a]==0 and base[b]==1:
                print(idx)
                new_base[a]=1
                new_base[b]=0
                
                jdx=basis.index(tuple(new_base))
                if jdx is not None:
                    matrix[jdx, idx]=1
        if a==b:
            if base[a]==1:
                matrix[idx,idx]=1
                
    return matrix

In [402]:
v_matrix=lil_matrix((len(valid_configs), len(valid_configs)), dtype=float)

for key, V in tbme_dict.items():
    (j1a,j1b), (j2a,j2b), J, T, Tz = key

    if (j1a,j1b,J,T,Tz) not in states_list:
        continue
    if (j2a,j2b,J,T,Tz) not in states_list:
        continue
    a=states_list.index((j1a,j1b,J,T,Tz))
    b=states_list.index((j2a,j2b,J,T,Tz))
    
    p_matrix=p_dag_p_matrix(a,b, valid_configs)
    if np.sum(p_matrix)!=0:
        print(a,b,p_matrix)
    v_matrix+=V*p_matrix

15 15 <List of Lists sparse matrix of dtype 'int64'
	with 9 stored elements and shape (70, 70)>
  Coords	Values
  (11, 11)	1
  (23, 23)	1
  (30, 30)	1
  (40, 40)	1
  (48, 48)	1
  (55, 55)	1
  (62, 62)	1
  (66, 66)	1
  (67, 67)	1
19 19 <List of Lists sparse matrix of dtype 'int64'
	with 9 stored elements and shape (70, 70)>
  Coords	Values
  (13, 13)	1
  (25, 25)	1
  (32, 32)	1
  (42, 42)	1
  (50, 50)	1
  (57, 57)	1
  (64, 64)	1
  (67, 67)	1
  (69, 69)	1
12 12 <List of Lists sparse matrix of dtype 'int64'
	with 5 stored elements and shape (70, 70)>
  Coords	Values
  (16, 16)	1
  (44, 44)	1
  (58, 58)	1
  (60, 60)	1
  (61, 61)	1
13 13 <List of Lists sparse matrix of dtype 'int64'
	with 9 stored elements and shape (70, 70)>
  Coords	Values
  (10, 10)	1
  (22, 22)	1
  (29, 29)	1
  (39, 39)	1
  (47, 47)	1
  (54, 54)	1
  (62, 62)	1
  (63, 63)	1
  (64, 64)	1
14 14 <List of Lists sparse matrix of dtype 'int64'
	with 5 stored elements and shape (70, 70)>
  Coords	Values
  (3, 3)	1
  (35, 35)	1


In [403]:
print(v_matrix.todense())

[[ 0.3397      0.          0.         ...  0.          0.
   0.        ]
 [ 0.          1.03285     0.         ...  0.          0.
   0.        ]
 [ 0.          0.         -0.97095    ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ... -0.49369373  0.
   0.        ]
 [ 0.          0.          0.         ...  0.         -3.11904801
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
  -4.89847401]]


In [404]:
h_spe_matrix=lil_matrix((len(valid_configs), len(valid_configs)), dtype=float)
for idx, base in enumerate(valid_configs):
    energy=0.0
    for bit, state in zip(base, states_list):
        if bit==1:
            j_a, j_b, J, T, Tz = state
            energy+=spe_per_j.get(j_a,0.0)+spe_per_j.get(j_b,0.0)
    h_spe_matrix[idx, idx]=energy
    
print(h_spe_matrix)

<List of Lists sparse matrix of dtype 'float64'
	with 70 stored elements and shape (70, 70)>
  Coords	Values
  (0, 0)	9.08
  (1, 1)	8.44
  (2, 2)	8.44
  (3, 3)	7.8
  (4, 4)	7.8
  (5, 5)	9.08
  (6, 6)	8.44
  (7, 7)	8.44
  (8, 8)	8.44
  (9, 9)	8.44
  (10, 10)	7.8
  (11, 11)	7.8
  (12, 12)	7.8
  (13, 13)	7.8
  (14, 14)	8.44
  (15, 15)	8.44
  (16, 16)	7.8
  (17, 17)	7.8
  (18, 18)	8.44
  (19, 19)	8.44
  (20, 20)	8.44
  (21, 21)	8.44
  (22, 22)	7.8
  (23, 23)	7.8
  (24, 24)	7.8
  :	:
  (45, 45)	7.16
  (46, 46)	7.8
  (47, 47)	7.16
  (48, 48)	7.16
  (49, 49)	7.16
  (50, 50)	7.16
  (51, 51)	7.8
  (52, 52)	7.16
  (53, 53)	7.16
  (54, 54)	7.16
  (55, 55)	7.16
  (56, 56)	7.16
  (57, 57)	7.16
  (58, 58)	7.16
  (59, 59)	7.16
  (60, 60)	6.52
  (61, 61)	6.52
  (62, 62)	6.52
  (63, 63)	6.52
  (64, 64)	6.52
  (65, 65)	6.52
  (66, 66)	6.52
  (67, 67)	6.52
  (68, 68)	6.52
  (69, 69)	6.52


In [ ]:
h=(v_matrix+v_matrix.T)+h_spe_matrix
print(h.todense())
e,psis=np.linalg.eigh(h.todense())
print(e)


[[ 9.7594      0.          0.         ...  0.          0.
   0.        ]
 [ 0.         10.5057      0.         ...  0.          0.
   0.        ]
 [ 0.          0.          6.4981     ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  5.53261253  0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.28190398
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
  -3.27694801]]
[-16.24111848 -15.10981848 -14.74071647 -12.17133697 -11.24806967
  -9.45314801  -7.9671      -7.59799798  -6.46669798  -6.09759597
  -6.06591848  -5.02861848  -3.89731848  -3.87615989  -3.52821647
  -3.52821647  -3.52821647  -3.2386205   -2.74485989  -2.37575787
  -2.31042953  -1.17912953  -0.95883697  -0.81002751  -0.81002751
  -0.81002751  -0.17883451   0.19362163   1.0768       1.75935199
   1.75935199   1.75935199   2.2081       2.57720202   2.57720202
   2.57720202   2.91181059   3.90409798   4.47754094   5.03539798
   5.14658152   5.14